create a dataset containing just stories of the CoQA dataset, for the sake of MLM.

In [14]:
!pip install datasets

     |████████████████████████████████| 306 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 51.5 MB/s 
     |████████████████████████████████| 67 kB 3.9 MB/s 
     |████████████████████████████████| 133 kB 64.8 MB/s 
     |████████████████████████████████| 1.1 MB 39.3 MB/s 
     |████████████████████████████████| 160 kB 74.5 MB/s 
     |████████████████████████████████| 144 kB 66.9 MB/s 
     |████████████████████████████████| 271 kB 63.9 MB/s 


In [27]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [22]:
from datasets import Dataset
import datasets

In [1]:
import requests

url_train = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
url_val = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"

json_train = requests.get(url_train).json()
json_val = requests.get(url_val).json()

In [2]:
samples_train = json_train['data']
samples_val = json_val['data']

In [3]:
len(samples_train), len(samples_val)

(7199, 500)

In [4]:
from pprint import pprint

pprint(samples_train[1])

{'answers': [{'input_text': 'Hard Rock Cafe',
              'span_end': 284,
              'span_start': 243,
              'span_text': "Hard Rock Cafe in New York's Times Square",
              'turn_id': 1},
             {'input_text': '$2 million.',
              'span_end': 210,
              'span_start': 180,
              'span_text': ' reaping a total $2 million. \n',
              'turn_id': 2},
             {'input_text': '$120,000',
              'span_end': 342,
              'span_start': 292,
              'span_text': ' pre-sale expectations of only $120,000 in sales. ',
              'turn_id': 3},
             {'input_text': 'Hoffman Ma',
              'span_end': 1365,
              'span_start': 1295,
              'span_text': 'Hoffman Ma, who bought the glove on behalf of '
                           'Ponte 16 Resort in Macau',
              'turn_id': 4},
             {'bad_turn': 'true',
              'input_text': 'Macau',
              'span_end': 1366,
      

In [37]:
from tqdm.notebook import tqdm

samples_train_list = []

for i, item in tqdm(enumerate(samples_train)):
    d = {
            "id": item["id"],
            "text": item["story"]
    }
    samples_train_list.append(d)

0it [00:00, ?it/s]

In [38]:
len(samples_train_list)

7199

In [39]:
from tqdm.notebook import tqdm

samples_val_list = []

for i, item in tqdm(enumerate(samples_val)):
    d = {
            "id": item["id"],
            "text": item["story"],
    }
    samples_val_list.append(d)

0it [00:00, ?it/s]

In [40]:
len(samples_val_list)

500

In [41]:
import pandas as pd

train_df = pd.DataFrame(data=samples_train_list)
val_df = pd.DataFrame(data=samples_val_list)

In [12]:
train_df

,id,text
0,3zotghdk5ibi9cex97fepx7jetpso7,"The Vatican Apostolic Library (), more commonl..."
1,3wj1oxy92agboo5nlq4r7bndc3t8a8,New York (CNN) -- More than 80 Michael Jackson...
2,3bdcf01ogxu7zdn9vlrbf2rqzwplyf,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""..."
3,3ewijtffvo7wwchw6rtyaf7mfwte0p,(CNN) -- The longest-running holiday special s...
4,3urfvvm165iantk80llvkwwbjs7uzh,CHAPTER XXIV. THE INTERRUPTED MASS \n\nThe mor...
...,...,...
7194,34j10vatjfyw0aohj8d4a0wwku3qif,"CHAPTER XX \n\nFAST IN THE ICE \n\n""Well, ther..."
7195,3vj40nv2qinjocrcy7k4z235g6aotn,(CNN) -- The biological mother of a missing 7-...
7196,3rjsc4xj10uw0to3vq0v6l191rz05q,"By the time Rihanna was seventeen ,she had rel..."
7197,3gs6s824sqxty8vusxp27xazutmnw2,"Frankfurt, officially Frankfurt am Main (Liter..."


In [42]:
train_df.to_pickle("coqa_flat_train_df.pkl")
val_df.to_pickle("coqa_flat_val_df.pkl")

In [43]:
ds_t = Dataset.from_pandas(train_df, split="train")
ds_v = Dataset.from_pandas(val_df, split="validation")

In [44]:
ds_t.push_to_hub('coqa-stories', split='train')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
ds = datasets.DatasetDict({
    'train': ds_t,
    'validation': ds_v
})

In [48]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 7199
    })
    validation: Dataset({
        features: ['id', 'text'],
        num_rows: 500
    })
})

In [49]:
ds.push_to_hub('coqa-stories')

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
ds = datasets.load_dataset('alistvt/coqa-stories')

Downloading:   0%|          | 0.00/722 [00:00<?, ?B/s]

Using custom data configuration alistvt--coqa-stories-02eb9f1d0742faf2


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/7.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/506k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/alistvt--coqa-stories-02eb9f1d0742faf2/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [51]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 7199
    })
    validation: Dataset({
        features: ['id', 'text'],
        num_rows: 500
    })
})

In [52]:
ds['train'][0]

{'id': '3zotghdk5ibi9cex97fepx7jetpso7',
 'text': 'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives wer

In [53]:
ds['validation'][1]

{'id': '3azhrg4cu4ktme1zh7c2ro3pn2430d',
 'text': 'Once there was a beautiful fish named Asta. Asta lived in the ocean. There were lots of other fish in the ocean where Asta lived. They played all day long. \n\nOne day, a bottle floated by over the heads of Asta and his friends. They looked up and saw the bottle. "What is it?" said Asta\'s friend Sharkie. "It looks like a bird\'s belly," said Asta. But when they swam closer, it was not a bird\'s belly. It was hard and clear, and there was something inside it. \n\nThe bottle floated above them. They wanted to open it. They wanted to see what was inside. So they caught the bottle and carried it down to the bottom of the ocean. They cracked it open on a rock. When they got it open, they found what was inside. It was a note. The note was written in orange crayon on white paper. Asta could not read the note. Sharkie could not read the note. They took the note to Asta\'s papa. "What does it say?" they asked. \n\nAsta\'s papa read the note. H